# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
# Group by 'id' and 'action' and count occurrences
view_click_counts = df.groupby(['id', 'action']).size().unstack(fill_value=0)
viewers_and_clickers = view_click_counts[(view_click_counts['view'] > 0) & (view_click_counts['click'] > 0)]

# Get the count of viewers who also clicked
num_viewers_clicked = len(viewers_and_clickers)
print(f"Number of viewers who also clicked: {num_viewers_clicked}")

Number of viewers who also clicked: 1860


In [4]:
# Find viewers who clicked but didn't view
anomalies = df[(df['action'] == 'click') & ~(df['id'].isin(viewers_and_clickers.index))]
print("Anomalies - Clicked but didn't view:")
print(anomalies)

Anomalies - Clicked but didn't view:
Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []


In [5]:
# Group by 'id' and 'group' and count occurrences
group_counts = df.groupby(['id', 'group']).size().unstack(fill_value=0)

# Check for overlap
overlap = group_counts[(group_counts['control'] > 0) & (group_counts['experiment'] > 0)]
print("Overlap between control and experiment groups:")
print(overlap)

Overlap between control and experiment groups:
Empty DataFrame
Columns: [control, experiment]
Index: []


There is no overlap between the control and experiment groups in the dataset. This is a positive outcome for experimental design, as it means that users are cleanly separated into their respective groups.

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
import pandas as pd
from scipy.stats import chi2_contingency

# Load the dataset
df = pd.read_csv("homepage_actions.csv")

# Create a contingency table of clicks by group
contingency_table = pd.crosstab(df['group'], df['action'])

# Perform the chi-squared test for independence
chi2, p, _, _ = chi2_contingency(contingency_table)

# Set significance level (alpha)
alpha = 0.05

# Check if the p-value is less than the significance level
if p < alpha:
    print(f"The experimental homepage was more effective than the control group (p-value={p}).")
else:
    print(f"There is no significant difference between the experimental and control groups (p-value={p}).")

There is no significant difference between the experimental and control groups (p-value=0.056537191086915774).


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [7]:
#Your code here
import pandas as pd

# Load the dataset
df = pd.read_csv("homepage_actions.csv")

# Calculate the click-through rate (CTR) for the control group
control_group = df[df['group'] == 'control']
control_ctr = control_group['action'].value_counts(normalize=True)['click']

# Calculate the number of observations in the experiment group
experiment_group = df[df['group'] == 'experiment']
num_experiment_observations = len(experiment_group)

# Calculate the expected number of clicks for the experiment group
expected_experiment_clicks = control_ctr * num_experiment_observations

print(f"Expected number of clicks for the experiment group: {expected_experiment_clicks}")

Expected number of clicks for the experiment group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [8]:
#Your code here
import pandas as pd
import math

# Load the dataset
df = pd.read_csv("homepage_actions.csv")

# Calculate the click-through rate (CTR) for the control group
control_group = df[df['group'] == 'control']
control_ctr = control_group['action'].value_counts(normalize=True)['click']

# Calculate the number of observations in the experiment group
experiment_group = df[df['group'] == 'experiment']
num_experiment_observations = len(experiment_group)

# Calculate the expected number of clicks for the experiment group
expected_experiment_clicks = control_ctr * num_experiment_observations

# Calculate the standard deviation of the binomial distribution
std_dev = math.sqrt(num_experiment_observations * control_ctr * (1 - control_ctr))

# Calculate the actual number of clicks in the experiment group
actual_experiment_clicks = experiment_group['action'].value_counts()['click']

# Calculate the z-score
z_score = (actual_experiment_clicks - expected_experiment_clicks) / std_dev

print(f"Number of standard deviations from the estimate: {z_score:.2f}")

Number of standard deviations from the estimate: 2.72


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
#Your code here
import pandas as pd
import math
from scipy.stats import norm

# Load the dataset
df = pd.read_csv("homepage_actions.csv")

# Calculate the click-through rate (CTR) for the control group
control_group = df[df['group'] == 'control']
control_ctr = control_group['action'].value_counts(normalize=True)['click']

# Calculate the number of observations in the experiment group
experiment_group = df[df['group'] == 'experiment']
num_experiment_observations = len(experiment_group)

# Calculate the expected number of clicks for the experiment group
expected_experiment_clicks = control_ctr * num_experiment_observations

# Calculate the standard deviation of the binomial distribution
std_dev = math.sqrt(num_experiment_observations * control_ctr * (1 - control_ctr))

# Calculate the actual number of clicks in the experiment group
actual_experiment_clicks = experiment_group['action'].value_counts()['click']

# Calculate the z-score
z_score = (actual_experiment_clicks - expected_experiment_clicks) / std_dev

# Calculate the p-value using the cumulative distribution function (CDF) of the standard normal distribution
p_value = 1 - norm.cdf(z_score)

print(f"Z-score: {z_score:.2f}")
print(f"P-value: {p_value:.4f}")

Z-score: 2.72
P-value: 0.0033


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your answer here**

The analysis using the binomial distribution and z-test suggests that the experimental homepage was statistically more effective than the control group, with a p-value of 0.0033, indicating a significant difference in click-through rates. This result aligns with the previous statistical test, with a lower p-value, reinforcing the conclusion that the experimental homepage had a positive impact.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.